In [1]:
import functools
import numpy as np
import tensorflow as tf
from keras.datasets import cifar100
from keras import Sequential
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from keras.optimizers import SGD
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D, LeakyReLU
from keras.layers.normalization import BatchNormalization
from tensorflow.keras.utils import to_categorical
from keras.utils.generic_utils import get_custom_objects

In [2]:
(x_train, y_train), (x_test, y_test) = cifar100.load_data()
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=.2)

In [3]:
y_train = to_categorical(y_train)
y_val = to_categorical(y_val)
y_test = to_categorical(y_test)

In [4]:
x_train.shape, y_train.shape, x_val.shape, y_val.shape, x_test.shape, y_test.shape

((40000, 32, 32, 3),
 (40000, 100),
 (10000, 32, 32, 3),
 (10000, 100),
 (10000, 32, 32, 3),
 (10000, 100))

In [5]:
train_generator = ImageDataGenerator(rotation_range=2, horizontal_flip=True)
val_generator = ImageDataGenerator(rotation_range=2, horizontal_flip=True)
test_generator = ImageDataGenerator(rotation_range=2, horizontal_flip=True)

train_generator.fit(x_train)
val_generator.fit(x_val)
test_generator.fit(x_test)

In [6]:
def alexnet(activation):
    AlexNet = Sequential()

    AlexNet.add(Conv2D(filters=96, input_shape=(32, 32, 3), kernel_size=(11, 11), strides=(4, 4), padding='same'))
    AlexNet.add(BatchNormalization())
    AlexNet.add(Activation(activation))
    AlexNet.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'))

    AlexNet.add(Conv2D(filters=256, kernel_size=(5, 5), strides=(1, 1), padding='same'))
    AlexNet.add(BatchNormalization())
    AlexNet.add(Activation(activation))
    AlexNet.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'))

    AlexNet.add(Conv2D(filters=384, kernel_size=(3, 3), strides=(1, 1), padding='same'))
    AlexNet.add(BatchNormalization())
    AlexNet.add(Activation(activation))

    AlexNet.add(Conv2D(filters=384, kernel_size=(3, 3), strides=(1, 1), padding='same'))
    AlexNet.add(BatchNormalization())
    AlexNet.add(Activation(activation))

    AlexNet.add(Conv2D(filters=256, kernel_size=(3, 3), strides=(1, 1), padding='same'))
    AlexNet.add(BatchNormalization())
    AlexNet.add(Activation(activation))
    AlexNet.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'))

    AlexNet.add(Flatten())
    AlexNet.add(Dense(4096, input_shape=(32, 32, 3)))
    AlexNet.add(BatchNormalization())
    AlexNet.add(Activation(activation))
    AlexNet.add(Dropout(0.5))

    AlexNet.add(Dense(4096))
    AlexNet.add(BatchNormalization())
    AlexNet.add(Activation(activation))
    AlexNet.add(Dropout(0.5))

    AlexNet.add(Dense(1000))
    AlexNet.add(BatchNormalization())
    AlexNet.add(Activation(activation))
    AlexNet.add(Dropout(0.5))

    AlexNet.add(Dense(100))
    AlexNet.add(BatchNormalization())
    AlexNet.add(Activation('softmax'))

    return AlexNet

In [7]:
def gelu(x):
    return 0.5 * x * (1 + tf.tanh(tf.sqrt(2 / np.pi) * (x + 0.044715 * tf.pow(x, 3))))
get_custom_objects().update({'gelu': Activation(gelu)})

def swish(x):
    return x * tf.sigmoid(x)
get_custom_objects().update({'swish': Activation(swish)})

get_custom_objects().update({'leaky-relu': Activation(LeakyReLU(alpha=0.2))})

# act_func = ['tanh', 'relu', 'leaky-relu', 'elu', 'selu', 'gelu', 'swish']

model = alexnet('tanh')

batch_size = 128
epochs = 50

top3_acc = functools.partial(tf.keras.metrics.top_k_categorical_accuracy, k=3)

top3_acc.__name__ = 'top3_acc'

model.compile(optimizer=SGD(), loss='categorical_crossentropy', metrics=['accuracy', top3_acc, 'top_k_categorical_accuracy'])

history = model.fit(train_generator.flow(x_train, y_train, batch_size=batch_size), epochs=epochs,
                      validation_data=val_generator.flow(x_val, y_val, batch_size=batch_size), verbose=1)

Epoch 1/50
313/313 [==============================] - 48s 88ms/step - loss: 4.7078 - accuracy: 0.0271 - top3_acc: 0.0708 - top_k_categorical_accuracy: 0.1104 - val_loss: 4.0470 - val_accuracy: 0.0909 - val_top3_acc: 0.2017 - val_top_k_categorical_accuracy: 0.2797
Epoch 2/50
313/313 [==============================] - 24s 78ms/step - loss: 4.0674 - accuracy: 0.0776 - top3_acc: 0.1855 - top_k_categorical_accuracy: 0.2657 - val_loss: 3.7407 - val_accuracy: 0.1351 - val_top3_acc: 0.2744 - val_top_k_categorical_accuracy: 0.3668
Epoch 3/50
313/313 [==============================] - 25s 80ms/step - loss: 3.8685 - accuracy: 0.1079 - top3_acc: 0.2369 - top_k_categorical_accuracy: 0.3274 - val_loss: 3.6091 - val_accuracy: 0.1624 - val_top3_acc: 0.3127 - val_top_k_categorical_accuracy: 0.4074
Epoch 4/50
313/313 [==============================] - 25s 81ms/step - loss: 3.7063 - accuracy: 0.1390 - top3_acc: 0.2892 - top_k_categorical_accuracy: 0.3846 - val_loss: 3.4713 - val_accuracy: 0.1803 - val_to

In [8]:
print(history.history)
# y_pred = np.argmax(model.predict(x_test), axis=1)
# y_true = np.argmax(y_test,axis=1)

# print(y_pred.shape)
# print(y_true.shape)

# print(np.sum(y_pred == y_true) / y_pred.shape[0])

print(model.evaluate(x_test, y_test))

{'loss': [4.469039440155029, 4.009764194488525, 3.82824969291687, 3.6856086254119873, 3.573978900909424, 3.473165512084961, 3.3792531490325928, 3.290339946746826, 3.215635061264038, 3.1420042514801025, 3.063722610473633, 2.9900567531585693, 2.9230830669403076, 2.8548388481140137, 2.790928363800049, 2.730712652206421, 2.6686155796051025, 2.599498987197876, 2.5348615646362305, 2.47955584526062, 2.415875196456909, 2.364344835281372, 2.3033337593078613, 2.247913122177124, 2.187262773513794, 2.134659767150879, 2.0670862197875977, 2.0229833126068115, 1.9624855518341064, 1.9028247594833374, 1.8489724397659302, 1.79710054397583, 1.747870922088623, 1.6912612915039062, 1.6423147916793823, 1.587158203125, 1.5395628213882446, 1.4819945096969604, 1.4209383726119995, 1.3794219493865967, 1.3224941492080688, 1.2776554822921753, 1.2360047101974487, 1.1822876930236816, 1.13877534866333, 1.0875763893127441, 1.0491057634353638, 1.0106574296951294, 0.9623174071311951, 0.9166955947875977], 'accuracy': [0.04